In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
df_location = session.table("NYC_TAXI_DB.GOLD.DIM_LOCATION")
df_location.show()


In [ ]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

df_loc = df_location.to_pandas()

print(df_loc.dtypes)

In [ ]:
# Streamlined version - only stores centroids and bounds
def streamlined_cleaning(df):
    # Convert to uppercase and clean
    df.columns = df.columns.str.upper()
    
    # Clean string columns
    df['ZONE_NAME'] = df['ZONE_NAME'].str.strip()
    df['BOROUGH'] = df['BOROUGH'].str.strip()
    
    # Convert ZONE_ID to int16
    #df['ZONE_ID'] = df['ZONE_ID'].astype('int16')
    #df['ZONE_NAME'] = df['ZONE_NAME'].astype('string')
    df['BOROUGH'] = df['BOROUGH'].astype('string')
    
    # Function to extract bounds and centroid directly
    def extract_geometry_info(wkt_string):
        try:
            coord_pattern = r'-?\d+\.\d+'
            matches = re.findall(coord_pattern, str(wkt_string))
            
            lons = []
            lats = []
            for i in range(0, len(matches), 2):
                if i + 1 < len(matches):
                    lons.append(float(matches[i]))
                    lats.append(float(matches[i + 1]))
            
            if lons and lats:
                return pd.Series({
                    'CENTROID_LON': sum(lons) / len(lons),
                    'CENTROID_LAT': sum(lats) / len(lats),
                    'MIN_LON': min(lons),
                    'MAX_LON': max(lons),
                    'MIN_LAT': min(lats),
                    'MAX_LAT': max(lats),
                    'NUM_COORDINATES': len(lons)
                })
            else:
                return pd.Series({
                    'CENTROID_LON': None, 'CENTROID_LAT': None,
                    'MIN_LON': None, 'MAX_LON': None,
                    'MIN_LAT': None, 'MAX_LAT': None,
                    'NUM_COORDINATES': 0
                })
                
        except Exception as e:
            print(f"Error processing geometry: {e}")
            return pd.Series({
                'CENTROID_LON': None, 'CENTROID_LAT': None,
                'MIN_LON': None, 'MAX_LON': None,
                'MIN_LAT': None, 'MAX_LAT': None,
                'NUM_COORDINATES': 0
            })
    
    # Apply geometry extraction
    geometry_info = df['ZONE_GEOM'].apply(extract_geometry_info)
    
    # Combine with original data
    result = pd.concat([df[['ZONE_ID', 'ZONE_NAME', 'BOROUGH']], geometry_info], axis=1)
    
    return result

# Use streamlined version
df_streamlined = streamlined_cleaning(df_loc.copy())

print("Streamlined version:")
print(df_streamlined.head())
print(f"\nData types: {df_streamlined.dtypes}")

# Save streamlined version


In [ ]:
df_snow = session.create_dataframe(df_streamlined)
df_snow.write.mode("overwrite").save_as_table("NYC_TAXI_DB.GOLD.DIM_LOCATION_CLEANED")
# pdf is your pandas DF


In [ ]:
from snowflake.snowpark.functions import col , row_number
from snowflake.snowpark.window import Window

fact = session.table("NYC_TAXI_DB.GOLD.FACT_TAXI_TRIPS")
pay  = session.table("NYC_TAXI_DB.GOLD.DIM_PAYMENT")
pas  = session.table("NYC_TAXI_DB.GOLD.DIM_PASSENGER")
datim  = session.table("NYC_TAXI_DB.GOLD.DIM_DATETIME")

df_sp = fact.join(pay, fact["payment_id"] == pay["payment_id"], "left").join(pas, fact["passenger_id"] == pas["passenger_id"], "left").join(datim, fact["PICKUP_TIME_ID"] == datim["DATETIME_ID"], "left")


# Convert only the necessary columns to pandas
df_joined = df_sp.select(
    "passenger_count",
    "trip_distance",
    "trip_duration",
    "fare_amount",
    "rate_code",
    "payment_type",
     "hour",
    "day_of_week",
    "month",
    "vendor_id",
).to_pandas()

df_joined.head()


In [ ]:
df_filtered = df_joined.iloc[2_000_000:].copy()


In [ ]:
df_filtered["TRIP_DISTANCE"] = df_filtered["TRIP_DISTANCE"].astype(float)
df_filtered["TRIP_DURATION"] = df_filtered["TRIP_DURATION"].astype(float)
df_filtered["FARE_AMOUNT"] = df_filtered["FARE_AMOUNT"].astype(float)

df_filtered["RATE_CODE"]    = df_filtered["RATE_CODE"].astype("category").cat.codes
df_filtered["PAYMENT_TYPE"] = df_filtered["PAYMENT_TYPE"].astype("category").cat.codes
df_filtered["VENDOR_ID"]    = df_filtered["VENDOR_ID"].astype("category").cat.codes


# Remove negative or impossible values
df_filtered = df_filtered[
    (df_filtered["TRIP_DISTANCE"] > 0) &
    (df_filtered["TRIP_DURATION"] > 0) &
    (df_filtered["FARE_AMOUNT"] > 0)
]


In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

X = df_filtered.drop("FARE_AMOUNT", axis=1)
y = df_filtered["FARE_AMOUNT"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = XGBRegressor()
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)

print(f"RMSE:{rmse} \nMAE: {mae} \nR2: {r2}")


In [ ]:
df_results = X_test.copy()
df_results["fare_test"] = y_test.values
df_results["fare_pred"] = y_pred

table_name = "ML_PREDICTIONS"
schema_name = "MARTS"

session.write_pandas(
    df_results,
    table_name,
    schema=schema_name,
    auto_create_table=True,
    overwrite=True
)